<a href="https://colab.research.google.com/github/TakahashixD/transfer-learning-dio/blob/main/transfer_learning_dio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [188]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [189]:
_URL = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'
path_to_zip = tf.keras.utils.get_file(origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'PetImages')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset, val_dataset = tf.keras.utils.image_dataset_from_directory(PATH,
                                                        validation_split = 0.2,
                                                        shuffle=True,
                                                        batch_size=BATCH_SIZE,
                                                        image_size=IMG_SIZE,
                                                        seed = 1,
                                                        subset = "both")




Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Using 5000 files for validation.


In [190]:
val_batches = tf.data.experimental.cardinality(val_dataset)
test_dataset = val_dataset.take(val_batches // 5)
val_dataset = val_dataset.skip(val_batches // 5)

In [191]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 126
Number of test batches: 31


In [192]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [193]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [194]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [195]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [196]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [197]:
base_model.trainable = False

In [198]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


In [199]:
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 1)


In [200]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [201]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])

In [202]:
val_dataset = val_dataset.ignore_errors()
train_dataset = train_dataset.ignore_errors()

In [203]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(val_dataset)

125/125 ━━━━━━━━━━━━━━━━━━━━ 78s 589ms/step - accuracy: 0.3798 - loss: 0.8943


In [204]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 0.89
initial accuracy: 0.39


In [206]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=val_dataset)

Epoch 1/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 500s 802ms/step - accuracy: 0.6890 - loss: 0.5716 - val_accuracy: 0.9630 - val_loss: 0.1391
Epoch 2/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 490s 794ms/step - accuracy: 0.9224 - loss: 0.2055 - val_accuracy: 0.9728 - val_loss: 0.0912
Epoch 3/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 500s 810ms/step - accuracy: 0.9382 - loss: 0.1638 - val_accuracy: 0.9764 - val_loss: 0.0749
Epoch 4/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 498s 806ms/step - accuracy: 0.9455 - loss: 0.1422 - val_accuracy: 0.9769 - val_loss: 0.0675
Epoch 5/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 486s 788ms/step - accuracy: 0.9473 - loss: 0.1320 - val_accuracy: 0.9786 - val_loss: 0.0623
Epoch 6/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 573s 928ms/step - accuracy: 0.9507 - loss: 0.1256 - val_accuracy: 0.9816 - val_loss: 0.0595
Epoch 7/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 676s 1s/step - accuracy: 0.9521 - loss: 0.1191 - val_accuracy: 0.9809 - val_loss: 0.0571
Epoch 8/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 509s 824ms/step - accuracy: 0.9530 - lo

In [208]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 625ms/step - accuracy: 0.9826 - loss: 0.0543
Test accuracy : 0.9838709831237793
